In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
from cmocean import cm
from cartopy.crs import LambertAzimuthalEqualArea
from cartopy.feature import LAND
from scipy.ndimage import gaussian_filter
from scipy.interpolate import RectBivariateSpline

%matplotlib inline

In [ ]:
ifile = '../mesh_arctic_ease_25km_max7.npz'
srs_dst = LambertAzimuthalEqualArea(central_longitude=0, central_latitude=90)

In [ ]:
with np.load(ifile) as ds:
    x = ds['x'] * 1000
    y = ds['y'] * 1000
    t = ds['t']
    xc = ds['xc'] * 1000
    yc = ds['yc'] * 1000
    mask = ds['mask']

In [ ]:
m = gaussian_filter(mask.astype(float), 1, truncate=1)
max_mask_fix_node=0.15
max_mask_zero_speed=0.6
mask = RectBivariateSpline(xc, yc, m[::-1], kx=1, ky=1)(y, x, grid=False)
fixed_nodes_idx = np.nonzero(mask < max_mask_fix_node)[0]
zero_speed = np.nonzero(mask < max_mask_zero_speed)[0]

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(1, 3, 1, projection=srs_dst)
tcf = ax.triplot(x, y, t, 'b-', lw=0.1, zorder=1)
f = ax.add_feature(LAND, facecolor='lightgray', edgecolor='black', zorder=0, linewidth=0.2)
ax.set_extent([-2200000, 2300000, -2500000, 2800000], crs=srs_dst)
ax.set_axis_on()
ax.set_xticks(ax.get_xticks())
ax.set_yticks(ax.get_yticks())

ax = fig.add_subplot(1, 3, 2, projection=srs_dst)
tcf = ax.triplot(x, y, t, 'b-', lw=0.1, zorder=1)
plt.plot(x[zero_speed], y[zero_speed], 'b.', markersize=2)
plt.plot(x[fixed_nodes_idx], y[fixed_nodes_idx], 'r.', markersize=2)
f = ax.add_feature(LAND, facecolor='lightgray', edgecolor='black', zorder=1, alpha=0.8, linewidth=0.2)
ax.set_extent([-1000000, 500000, -2000000, -200000], crs=srs_dst)
ax.set_axis_on()
ax.set_xticks(ax.get_xticks())
ax.set_yticks(ax.get_yticks())

ax = fig.add_subplot(1, 3, 3, projection=srs_dst)
tcf = ax.triplot(x, y, t, 'b-', lw=0.1)
plt.plot(x[zero_speed], y[zero_speed], 'b.', markersize=2)
plt.plot(x[fixed_nodes_idx], y[fixed_nodes_idx], 'r.', markersize=2)
f = ax.add_feature(LAND, facecolor='lightgray', edgecolor='black', zorder=1, alpha=0.8, linewidth=0.2)
ax.set_extent([-80000, 800000, -1700000, -700000], crs=srs_dst)
ax.set_axis_on()
ax.set_xticks(ax.get_xticks())
ax.set_yticks(ax.get_yticks())

plt.tight_layout()
plt.savefig('figure04_mesh.png', dpi=100, bbox_inches='tight', pad_inches=0.1)